In [3]:
# Imports 
import matplotlib.pyplot as plt
import numpy as np

In [11]:
def round_grade(grade):
    return grade+5-grade%5 if grade >= 38 and grade%5 > 2 else grade 
grades=[73,67,38,33]

round_grade(38)

38